In [934]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import string
import math


So, I am pretty new to Machine Learning (my only background is the Online Coursera Course from Andrew Ng), so this is my plan for this project. 

1. I am going to first analyze the data shown in order to get a better feel of the data itself and what its structure is. I will do this using pandas and matplot lib. Relevant figures will be shown in Feature Analysis Document. Additionally, I will update the data by creating new features that would help in computation.

2. After analyzing features, I will create two machine learning models from scratch, a decision tree, trained and tested recursively using the Gini Impurity metric as well as a logistic regression model. These two models will help me predict the titanic result for the test data. 

3. In each model I will try to avoid overfitting, this will be done through regularization in the case of logistic regression, and by creating a min node cap for the decision tree. 

4. I will split the training data into two sets, Regular and Cross-Validation, using a 80 - 20 split. After evaluating the data on each set, using the F-score metric, I will tweak the regularization terms accordingly in order to produce the best results. I will also compare my results (for logistic regression) to well known logistic regression library, sklearn. 

5. Finally, I will do the same to the test data and submit my scores on Kaggle.


All of the models will be shown in the code below. However, if you wish to check out how features were analyzed, you may look at the feature analysis document. 

In [935]:
#f-score Metrix

def calc_f_score(y_pred, y_test):
    true_positive = 0
    false_negatives = 0
    false_positives = 0
    for i in range(len(y_pred)):
        if y_pred[i] == 1 and y_test[i] == 1:
            true_positive += 1
        elif y_pred[i] == 1 and y_test[i] == 0:
            false_positives += 1
        elif y_pred[i] == 0 and y_test[i] == 1:
            false_negatives += 1
    precision = true_positive / (true_positive + false_positives)
    recall = true_positive/ (true_positive + false_negatives)
    return  2 / (1/precision + 1/recall)   

#from Feature Analysis'
data = pd.read_csv('Titanic/train.csv')

def avg(lst):
    return sum(lst)/ len(lst)

def change_sex_to_number(gender):
    if gender == 'male':
        return 0
    else:
        return 1

def change_embarked_to_number(embarked):
    if embarked == 'S':
        return 1
    elif embarked == 'C':
        return 2
    elif embarked == 'Q':
        return 3
    else:
        return 0
    
def add_feat(data):
    
    titles = ['Mrs','Mr', 'Miss', 'Ms', 'Master', 'Dona', 'Captain', 'Rev', 'Don', 'Dr', 'Major', 'Mlle', 'Col', 'Jonkheer', 'Mme', 'Capt', 'Countess' , 'Sir', 'Mme'] 

    simp_title = {'Mr': 1, 'Mrs': 2, 'Master': 3, 'Miss': 4, 'Army': 5, 'Academic': 6, 'Royal': 7}
            
    data['Sex_Number'] = [change_sex_to_number(sex) for sex in data['Sex']]
    data['Embarked_number'] = [change_embarked_to_number(embarked) for embarked in data['Embarked']]
    data['Name_Plain'] = ["".join(l for l in b if l not in string.punctuation) for b in data['Name']]
    data['Very_Young'] = [int(b <= 5) for b in data['Age']]
    data['Title'] = [[word for word in sting.split() if word in titles][0] for sting in data['Name_Plain']]
    data['Title'] = data['Title'].replace(['Capt', 'Col', 'Major'], 'Army')
    data['Title'] = data['Title'].replace(['Dr', 'Rev'], 'Academic')
    data['Title'] = data['Title'].replace(['Countess', 'Lady', 'Sir', 'Jonkheer', 'Don', 'Dona'], 'Royal')
    data['Title'] = data['Title'].replace(['Mlle', 'Ms'], 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
    data['Title_Numeric'] = [simp_title[sting] for sting in data['Title']]



def change_age(data):
    nul = data['Age'].isnull()
    old_male_avg = avg([data['Age'][index] for index in range(len(data['Age'])) if not nul[index] and data['Title_Numeric'][index] == 1])
    
    old_female_avg = avg([data['Age'][index] for index in range(len(data['Age'])) if not nul[index] and data['Title_Numeric'][index] == 2])
    
    young_female_avg = avg([data['Age'][index] for index in range(len(data['Age'])) if not nul[index] and data['Title_Numeric'][index] == 4])
    young_male_avg = avg([data['Age'][index] for index in range(len(data['Age'])) if not nul[index] and data['Title_Numeric'][index] == 3])
    army_average = avg([data['Age'][index] for index in range(len(data['Age'])) if not nul[index] and data['Title_Numeric'][index] == 5])
    royal_average =avg([data['Age'][index] for index in range(len(data['Age'])) if not nul[index] and data['Title_Numeric'][index] == 7])
    academic_average = avg([data['Age'][index] for index in range(len(data['Age'])) if not nul[index] and data['Title_Numeric'][index] == 6])
    age_avg = { 1: old_male_avg, 2: old_female_avg, 4: young_female_avg, 3: young_male_avg, 5: army_average, 6: academic_average, 7: royal_average}
    
   
    
    
    for index in range(len(nul)):
        if nul[index]:
            if data['Title_Numeric'][index] in age_avg:
                data['Age'][index] = age_avg[data['Title_Numeric'][index]]
      

                

    
#From Decision Tree

class DecisionTree:
    empty_positive = [1]
    empty_negative = [0]
    empty = []
    threshold = 0.05
    def __init__(self, total = 1):
        self.right = self.empty
        self.left  = self.empty
        DecisionTree.threshold = DecisionTree.threshold * total
    def __str__(self):
        if self.left == self.empty_positive:
            left = '(+)'
        elif self.left == self.empty_negative:
            left = '(-)'
        else:
            left = str(self.left)
        if self.right == self.empty_positive:
            right = '(+)'
        elif self.right == self.empty_negative:
            right = '(-)'
        else:
            right = str(self.right)
        return '(' + str(self.split) + ', ' + str(self.col) + ')\n' + left + '\n' + right
    
   
    
    def test(self, data_pt):
        assert self is not self.empty
        if data_pt[self.col] < self.split:
            if self.right == self.empty_positive:
                return 1
            elif self.right == self.empty_negative:
                return 0
            else:
                return self.right.test(data_pt)
        else:
            if self.left == self.empty_positive:
                return 1
            elif self.left == self.empty_negative:
                return 0
            else:
                return self.left.test(data_pt)
    
    def calculate_gini_impurity(self, X, y, split, col):
        p_right = 0
        num_right = 0
        p_left = 0
        num_left = 0
        #print('Col: ', col)
        
        for index in range(len(X[:, col])):
                      
            if X[index, col] > split:
                num_right += 1
                if y[0, index] != 0:
                    p_right += 1
            else:
                num_left += 1
                
                if y[0, index] != 0:
                    p_left += 1
                
        
        
        p_left = p_left / num_left
        p_right = p_right / num_right
        #print('PLeft: ', p_left)
        #print('PRight: ', p_right)
        
        total_impurity = ((p_left - p_left*p_left) * num_left + (p_right - p_right * p_right) * num_right)/ (num_left + num_right)
        #print('Total Impurity: ', total_impurity)
        
        return total_impurity
            
        
        
        
    def train(self, X_and_y):
        self.matrix = X_and_y
        X = np.matrix(X_and_y[:, :-1])
        
        y = np.matrix(X_and_y[:, -1])
        
        n, m = np.shape(X)
        col_number = 0
        split_value= 0
        min_impur = 1
        for col in range(m):
            mini = min(X[:, col])
            maxi = max(X[:, col])
            interval = (maxi - mini)/10
            if interval == 0:
                continue
            for _ in range(10):
                
                impur = self.calculate_gini_impurity(X, y, mini[0,0], col)
                
                if impur <= min_impur:
                    min_impur = impur
                    col_number = col
                    split_value = mini[0,0]
                mini += interval
            
        self.split = split_value
        self.col = col_number
        
        right_matrix = X_and_y[ (X_and_y[:, col_number] < self.split).ravel(), :]
        left_matrix = X_and_y[ (X_and_y[:, col_number] >= self.split).ravel(), :]
        
        
        if len(right_matrix) > self.threshold and len(left_matrix) > 0: 
            self.right = DecisionTree()
            self.right.train(right_matrix)
        else:
            if np.mean(right_matrix[:, -1]) < 0.5:
                self.right = self.empty_negative
            else:
                self.right = self.empty_positive
        if len(left_matrix) > self.threshold and len(right_matrix) > 0:
            self.left = DecisionTree()
            self.left.train(left_matrix)
        else:
            if np.mean(left_matrix[:, -1]) < 0.5:
                self.left = self.empty_negative
            else:
                self.left = self.empty_positive
            
        
#From LogisticRegression


def square(x):
        return x*x
    
class LogisticRegression:
    
    def __init__(self, regularized = True):
        self.regularized = regularized
    def sigmoid(self, x):
        return 1/(1 + np.exp(-1 * x))
    def logistic_function(self, vec, theta):
        return self.sigmoid(np.dot(vec,theta))
    
    def single_value_cost(self, x_vector, theta, y_value):
        x_vector = np.append(x_vector, 1)
        
        return y_value *  np.log(self.logistic_function(x_vector, theta)) + (1 - y_value) *  np.log( 1 - self.logistic_function(x_vector, theta))
    def gradient(self,theta, X, y):  
        theta = np.matrix(theta)
        X = np.matrix(X)
        y = np.matrix(y)
        n,m = X.shape # for generality
        X0 = np.ones((n,1))
        X= np.hstack((X,X0))
        y = y.T
        parameters = int(theta.ravel().shape[1])
        grad = np.zeros(parameters)
        
        error = self.sigmoid(X * theta.T) - y
        
        for i in range(parameters):
            term = np.multiply(error, X[:,i])
            grad[i] = np.sum(term) / len(X)

        return grad 
    def reg_gradient(self, X, y, theta, lamb):
        
        return self.gradient(theta, X, y) + lamb/(len(y)) * np.append(np.array(theta[:-1]), [0])

    def cost_function(self, x, theta, y):
        m = len(y)
        return (-1/(m)) * sum([self.single_value_cost(x[i], theta, y[i]) for i in range(m)])
    
    def reg_cost_function(self, x, theta, y, lamb):
        
        return self.cost_function(x, theta, y) + lamb/(2 *len(y)) * np.dot(theta[:-1], theta[:-1])
    
    def grad_check(self, x, y, index, theta, lamb):
        
        func_before = self.reg_cost_function(x, theta, y, lamb)
        theta[index] += 0.0000001
        func_after = self.reg_cost_function(x, theta, y, lamb)
        theta[index] -= 0.0000001
        return (func_after- func_before) / 0.0000001
    
    def update_theta(self, theta, alpha, x, y, lamb):
        
        gradient = self.reg_gradient(x, y, theta, lamb)
        
        #check_gradient = [self.grad_check(x, y, index, theta, lamb) for index in range(len(theta))]
        
        #print(gradient)
        
        #print('Check Gradient: ', check_gradient)
        #print('Gradient: ', gradient)
        
        theta = np.array(theta) - alpha * np.array(gradient)
        
        return theta
    
    def initialize_theta(self, x):
        return [rd.random() for i in range(len(x[0]) + 1)]
    
    def scale(self, x):
        x = np.array(x)
        
        x_normed = (x - self.mean)/(self.std)
        
        return x_normed
    
    def train(self, x, y, alpha, num_times, lamb):
        
        x = np.array(x)
        y = np.array(y)
        self.mean = x.mean(axis = 0)
        self.std = x.std(axis = 0)
        new_x= self.scale(x)
        theta = self.initialize_theta(new_x)
        prev = self.reg_cost_function(new_x, theta, y, lamb)
        while num_times > 0:
            if num_times % 100 == 0:
                print('Cost Function Value: ', self.reg_cost_function(new_x, theta, y, lamb))
                
            theta = self.update_theta(theta, alpha, new_x, y, lamb)
            curr = self.reg_cost_function(new_x, theta, y, lamb)
            
            prev = curr
            num_times -= 1
            
        print(theta)
        
        
        self.theta = theta
        #print(self.test(x))
    #def regularized_train(self, x, y, alpha, num_times, lamb):
    
    
    def test(self, x):
        new_x = self.scale(x)
        return np.matrix([np.dot(self.theta, np.matrix(np.append(x_vector, 1)).T) for x_vector in new_x]) 
                    
    


In [936]:
#splitting data into training and cv sets
add_feat(data)
change_age(data)
print(data)
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]


/Users/aagarwal_601/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25         0       3   
25          

In [937]:
print(len(test))

178


In [938]:
print(len(train))

713


In [939]:
#Gathering relevant information from training/testing dataframe
def split(data):
    things_in_x = ['Pclass', 'Age', 'Fare', 'Parch', 'Sex_Number', 'Embarked_number', 'Title_Numeric', 'Very_Young']
    x = np.array([np.array(data[thing]) for thing in things_in_x])
    y = np.array(data['Survived'])
    return x, y

x_train, y = split(train)
x = np.matrix(x_train).T



In [940]:
#Logistic Regression
log = LogisticRegression()
log.train(x, y, 1, 1000, 5)

Cost Function Value:  0.661375367592
Cost Function Value:  0.451914967153
Cost Function Value:  0.451913471314
Cost Function Value:  0.45191347129
Cost Function Value:  0.45191347129
Cost Function Value:  0.45191347129
Cost Function Value:  0.45191347129
Cost Function Value:  0.45191347129
Cost Function Value:  0.45191347129
Cost Function Value:  0.45191347129
[-0.87929817 -0.2443481   0.08078627 -0.15402218  1.20171234  0.19264687
 -0.03744065  0.35578375 -0.60760625]


In [941]:
#Testing
x_test, y_test = split(test)
x_test = np.matrix(x_test).T
k = log.scale(x_test)

In [942]:
y_pred = []
for vec in k:
    l = np.append(vec, 1)
    y_pred += [int(round(log.sigmoid(np.dot(log.theta, l))))]

     
print(calc_f_score(y_pred, y_test))
#THIS IS THE ACCURACY OF  THE MODEL

0.768


In [943]:
#Built-in Logistic Regression Model
from sklearn.linear_model import LogisticRegression as lr
from sklearn import metrics
logreg = lr()
logreg.fit(x, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [944]:
y_pred0 = logreg.predict(x_test)

In [945]:
print(y_pred0)

[0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1
 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1
 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 1
 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0]


In [946]:
print(y_test)

[0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 0
 1 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1
 0 0 0 0 1 1 1 1 0 0 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 0 0 0 1 1 0 1 1 0 0
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1
 1 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0]


In [947]:
calc_f_score(y_pred, y_test)
#F-score of built in model


0.768

It looks like the model I implemented from scratch not only matches the predictions by the sk.learn model, but it acutally seems to improve on them in the cross-validation. This suggests that my algorithmic implementation of logistic regression is correct. Now on to the decision tree 

In [948]:
#Splits data into information relevant for tree testing
def split_tree(data):
    things_in_x = ['Pclass', 'Age', 'Fare', 'Parch', 'Sex_Number', 'Embarked_number', 'Title_Numeric', 'Very_Young', 'Survived']
    x_and_y = data[things_in_x]
    return x_and_y.as_matrix()
x_and_y = split_tree(train)
print(x_and_y)



[[  3.          22.           7.25       ...,   1.           0.           0.        ]
 [  1.          38.          71.2833     ...,   2.           0.           1.        ]
 [  3.          26.           7.925      ...,   4.           0.           1.        ]
 ..., 
 [  3.          21.84563758  23.45       ...,   4.           0.           0.        ]
 [  1.          26.          30.         ...,   1.           0.           1.        ]
 [  3.          32.           7.75       ...,   1.           0.           0.        ]]


In [949]:

print(len(x_and_y))

713


In [950]:
#Decision Tree Training
tree = DecisionTree(len(x_and_y))
tree.train(x_and_y)

In [951]:
print(tree)

(1.6, 6)
(2.8, 0)
(19.31, 2)
(0.6, 3)
(25.14625, 2)
(-)
(-)
(+)
(6.678, 1)
(8.00083, 2)
(1.8, 3)
(+)
(0.9, 4)
(-)
(+)
(+)
(+)
(4.5, 6)
(-)
(1.8, 5)
(48.2, 1)
(+)
(0.9, 7)
(+)
(3.8, 6)
(+)
(+)
(136.75, 2)
(+)
(35.13, 2)
(+)
(23.602, 1)
(27.4, 1)
(+)
(+)
(+)
(2.0, 0)
(51.45, 2)
(-)
(23.8125, 2)
(-)
(2.0, 5)
(15.555, 2)
(+)
(29.3, 1)
(2.9, 0)
(8.25, 2)
(-)
(-)
(+)
(-)
(13.515, 2)
(-)
(9.45, 2)
(29.8, 1)
(-)
(-)
(27.6, 1)
(6.93, 2)
(8.01582, 2)
(-)
(7.74916, 2)
(-)
(-)
(-)
(23.7, 1)
(-)
(21.6, 1)
(-)
(-)
(48.5, 1)
(-)
(32.5, 1)
(+)
(131.5, 2)
(-)
(-)


In [952]:
print([tree.test(pt) for pt in split_tree(test)])

[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]


In [953]:
y_pred2 = [tree.test(pt) for pt in split_tree(test)]
len(y_pred2)

178

In [954]:
len(y_test)


178

In [955]:
calc_f_score(y_pred2, y_test)

0.694214876033058

All In All, we see that regardless of which method we use we obtain similar results. Specificially a 70-75% accuracy on this data set. Repeating Multiple Trials, suggests that both methods are fairly equal in their assessment of the data. However, logistic regression had a better result, so I will use that model. 

In [970]:
data_test = pd.read_csv('Titanic/test.csv')
add_feat(data_test)
change_age(data_test)
print(data_test)


/Users/aagarwal_601/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     PassengerId  Pclass                                               Name  \
0            892       3                                   Kelly, Mr. James   
1            893       3                   Wilkes, Mrs. James (Ellen Needs)   
2            894       2                          Myles, Mr. Thomas Francis   
3            895       3                                   Wirz, Mr. Albert   
4            896       3       Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
5            897       3                         Svensson, Mr. Johan Cervin   
6            898       3                               Connolly, Miss. Kate   
7            899       2                       Caldwell, Mr. Albert Francis   
8            900       3          Abrahim, Mrs. Joseph (Sophie Halaut Easu)   
9            901       3                            Davies, Mr. John Samuel   
10           902       3                                   Ilieff, Mr. Ylio   
11           903       1                         Jon

In [991]:
data_test.isnull().sum()
#quick replacement of null value in fare with average fare value
data_test['Fare'] = data_test['Fare'].replace(np.NaN, np.mean(data_test['Fare']))
data_test['Survived'] = data_test['PassengerId']

In [992]:
data_test.isnull().sum()


PassengerId          0
Pclass               0
Name                 0
Sex                  0
Age                  0
SibSp                0
Parch                0
Ticket               0
Fare                 0
Cabin              327
Embarked             0
Sex_Number           0
Embarked_number      0
Name_Plain           0
Very_Young           0
Title                0
Title_Numeric        0
Survived             0
dtype: int64

In [993]:
test, nothing = split(data_test)
test = np.matrix(test).T
test = log.scale(test)

In [997]:
y_pred = []
for vec in test:
    l = np.append(vec, 1)
    y_pred += [int(round(log.sigmoid(np.dot(log.theta, l))))]

In [999]:
print(y_pred)

[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 